In [116]:
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.tsa.seasonal as tsa
import statsmodels.api as sm
import datetime as dt
import numpy as np

from lets_plot import *
LetsPlot.setup_html()
from IPython.display import Image
import cairosvg

## Taylor Rule Deviation:

$i_t = 2 + \pi_t + a(\pi_t-\pi^*)+b(y_t - y_t^*)$

+ From the formula above we need:

    + Interest Rate
    + Inflation Rate
    + Targetted Inflation Rate
    + Potential Output

### Interest Rates:

Sources: https://iftp.chinamoney.com.cn/english/bmklpr/, https://www.bis.org/statistics/cbpol/cbpol.xlsx

I need to quarterlyize the interest rates

In [117]:
def strpquarter(i):
    i = str(i)
    if i[4:6] == 'Q1':
        quarter = '01'
    elif i[4:6] == 'Q2':
        quarter = '04'
    elif i[4:6] == 'Q3':
        quarter = '07'
    elif i[4:6] == 'Q4':
        quarter = '10'
    i = i[0:4] + '-' + quarter
    return pd.to_datetime(i)

In [118]:
# Nominal GDP
ustar = 0.05
df = pd.read_csv('../data/us/nominalgdp.csv', index_col='observation_date')
df = df.rename(columns={'NGDPSAXDCUSQ':'NominalGDP'})
df.index = pd.to_datetime(df.index)
df.index.names = ['QuarterDate']

# Interest Rate
df1 = pd.read_csv('../data/us/fedfundsrate.csv')
df1['Quarter'] = pd.PeriodIndex(df1['observation_date'],freq = 'Q')
df1 = df1[['FEDFUNDS','Quarter']].groupby('Quarter', as_index=False).mean()
df1['QuarterDate'] = df1['Quarter'].apply(strpquarter)
df1['QuarterDate'] = pd.to_datetime(df1['QuarterDate'])
df1 = df1.set_index(df1['QuarterDate'])
df1 = df1.drop(columns = ['QuarterDate'])
df['Interest'] = df1['FEDFUNDS']/100

# Unemployment Rate
df1 = pd.read_csv('../data/us/unemployment.csv', index_col='observation_date')
df1.index = pd.to_datetime(df1.index)
df['UnemploymentRate'] = df1['UNRATE']/100

# Primary Expenditure
df1 = pd.read_excel('../data/us/primaryexp.xls')
df1 = df1.transpose()
df1.index = pd.to_datetime(df1.index, format = '%Y')
df['PrimExp'] = -df1[0]/100
df['PrimExp'] = df['PrimExp'].fillna(method='ffill')

# Inflation
df1 = pd.read_csv('../data/us/inflation.csv', index_col='observation_date')
df1.index = pd.to_datetime(df1.index)
df['Inflation'] = df1['CPALTT01USQ657N']/100

# Debt
df1 = pd.read_csv('../data/us/debt.csv', index_col='observation_date')
df1.index = pd.to_datetime(df1.index)
df['Debt'] = df1['GFDEBTN']/100
df['DebtMinusHalf'] = df['Debt'].shift(2)
df.loc['2010-01-01':'2010-04-01','DebtMinusHalf'] = df.loc['2010-07-01','DebtMinusHalf']


# Output Gap
df['Ystar'] = df['NominalGDP']/(1-0.4*(df['UnemploymentRate']-ustar))
df['OutputGap'] = (df['NominalGDP']-df['Ystar'])/df['Ystar']
df['Istar'] = (2 + 1.5*df['Inflation'] + 0.5*df['OutputGap']+1)/100

# Taylor Deviation
df['TaylorDeviation'] = df['Interest']-df['Istar']

df

C:\Users\ellem\AppData\Local\Temp\ipykernel_4808\561408246.py:28: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['PrimExp'] = df['PrimExp'].fillna(method='ffill')


,NominalGDP,Interest,UnemploymentRate,PrimExp,Inflation,Debt,DebtMinusHalf,Ystar,OutputGap,Istar,TaylorDeviation
QuarterDate,,,,,,,,,,,
2010-01-01,3691152.50,0.001333,0.098,-0.370845,0.004014,127731.23,127731.23,3.763410e+06,-0.0192,0.029964,-0.028631
2010-04-01,3745048.25,0.001933,0.099,-0.370845,0.004751,132017.92,127731.23,3.819919e+06,-0.0196,0.029973,-0.028040
2010-07-01,3785401.75,0.001867,0.094,-0.370845,0.000933,135616.22,127731.23,3.853218e+06,-0.0176,0.029926,-0.028059
2010-10-01,3827368.50,0.001867,0.094,-0.370845,0.002949,140252.15,132017.92,3.895937e+06,-0.0176,0.029956,-0.028090
2011-01-01,3837862.00,0.001567,0.091,-0.359164,0.012648,142701.14,135616.22,3.901852e+06,-0.0164,0.030108,-0.028541
2011-04-01,3889384.75,0.000933,0.091,-0.359164,0.017433,143430.87,140252.15,3.954234e+06,-0.0164,0.030179,-0.029246
2011-07-01,3911920.00,0.000833,0.090,-0.359164,0.004085,147903.40,142701.14,3.975528e+06,-0.0160,0.029981,-0.029148
2011-10-01,3960564.75,0.000733,0.088,-0.359164,-0.001521,152229.40,143430.87,4.021695e+06,-0.0152,0.029901,-0.029168
2012-01-01,4017201.25,0.001033,0.083,-0.344963,0.007956,156065.18,147903.40,4.070938e+06,-0.0132,0.030053,-0.029020


In [119]:
plot = df.copy()
plot['Ystar'] = plot['NominalGDP']/((1-0.4*(df['UnemploymentRate']-ustar)))
plot4 = (
    ggplot(data = plot.reset_index(), mapping = aes(x = 'QuarterDate'))
    + geom_line(aes(y = 'NominalGDP'), manual_key = 'GDP', color = 'red')
    + geom_line(aes(y = 'Ystar'), manual_key = 'Potential GDP', color = 'black')
    + theme(legend_position = 'bottom',
            plot_caption=element_text(color='grey',size=9,hjust=0),
            plot_caption_position='panel'
            )
    + labs(y = 'Output',
           x='Date',
           title = '',
           caption = f"""Nominal GDP has been seasonally adjusted from the original data set. Since potential output is a function of unemployment 
           due to being calculated through Okun's law, output potential indirectly illustrates the effect of unemployment on the rest of 
           the variables.""")
    + scale_y_continuous(format = 's')
)
ggsave(plot4, 'plot4.png', path= '../images')
plot4

In [120]:
formula = 'PrimExp ~ DebtMinusHalf + OutputGap'
model1 = smf.ols(formula = formula, data = df).fit()
print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:                PrimExp   R-squared:                       0.625
Model:                            OLS   Adj. R-squared:                  0.606
Method:                 Least Squares   F-statistic:                     34.09
Date:                Wed, 05 Mar 2025   Prob (F-statistic):           1.90e-09
Time:                        14:06:05   Log-Likelihood:                 116.78
No. Observations:                  44   AIC:                            -227.6
Df Residuals:                      41   BIC:                            -222.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -0.2438      0.018    -13.479

In [121]:
plot = df.copy()
plot['PrimExp'] = plot['PrimExp']/3 + 0.1
plot['Debt'] = plot['Debt']
plot5 = (
    ggplot(data = plot.reset_index(), mapping = aes(x = 'QuarterDate'))
    + geom_line(aes(y = 'OutputGap'), manual_key = 'Output Gap', color = 'red')
    + geom_line(aes(y = 'PrimExp'), manual_key = 'Primary Expenditure')
    + theme_classic()
    + theme(legend_position='top',
            plot_title=element_text(face='bold'),
            axis_text_y = element_blank(),
            plot_subtitle = element_text(color = 'grey'),
            plot_caption = element_text(color='grey')
            )
    + labs(y = 'Normalized Axis', 
           x = '',
           title = 'Primary Expenditure (Measured) and Output Gap (Calculated) Covary with one Another',
           subtitle = 'Their positive covariance also suggests that US fiscal spending is slightly counter-cyclical.',
           caption = 'Axes are exaggerated for clearer illustration of covariance.'
           )
    + ggsize(1000,500)
    
)
ggsave(plot5, 'plot5.png', path= '../images')
plot5

In [122]:
formula = 'Interest ~ Inflation + OutputGap'
model = smf.ols(formula = formula, data = df).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:               Interest   R-squared:                       0.447
Model:                            OLS   Adj. R-squared:                  0.420
Method:                 Least Squares   F-statistic:                     16.57
Date:                Wed, 05 Mar 2025   Prob (F-statistic):           5.32e-06
Time:                        14:06:05   Log-Likelihood:                 166.42
No. Observations:                  44   AIC:                            -326.8
Df Residuals:                      41   BIC:                            -321.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0086      0.001      6.936      0.0

In [127]:
plot = df.copy()
plot['PrimExp'] = plot['PrimExp']/5 + 0.05
plot6 = (
    ggplot(data = plot.reset_index(), mapping = aes(x = 'QuarterDate'))
    + geom_line(aes(y = 'Interest'), manual_key = 'Federal Funds Rate', color = 'blue')
    + geom_line(aes(y = 'PrimExp'), manual_key = 'Primary Expenditure')
    + theme_classic()
    + labs(x = '', 
           y = 'Normalized Axis', 
           title = 'The Federal Funds Rate and Primary Expenditure Show Low Positive Covariance', 
           subtitle = 'This signifies that US Monetary Policy and US Fiscal Policy do work Hand-in-hand, but less so than China.',
           caption = f'Primary Expenditure and Federal Funds Rate have a correlation coefficient of {np.corrcoef(df['PrimExp'],df['Interest'])[0,1]}')
    + theme(plot_title = element_text(face = 'bold'),
            plot_subtitle = element_text(color = 'grey'),
            legend_position='top',
            plot_caption = element_text(color='grey'),
            axis_text_y= element_blank()
            )
    + ggsize(1000,500)
)
ggsave(plot6, 'plot6.png', path= '../images')
plot6